In [1]:
from cafe_sim_config import CafeSimulationConfigUS

cfg = CafeSimulationConfigUS()
cfg.create_ui()

Accordion(children=(VBox(children=(IntSlider(value=10, description='Příchody/hod:', layout=Layout(width='500px…

Button(button_style='success', description='Uložit do YAML', icon='save', style=ButtonStyle())

In [ ]:
print(cfg["num_cashiers"])

2
Konfigurace byla úspěšně uložena do: c:\Users\Michal Šeda\Projects\XMOSM\semestral_paper\us_default_config.yaml
